# Day 09. Exercise 02
# Metrics

## 0. Imports

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import pandas as pd

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [2]:
df = pd.read_csv('../../datasets/day-of-week-not-scaled.csv')
df_scaled = pd.read_csv('../../datasets/dayofweek.csv')
df['dayofweek'] = df_scaled['dayofweek']
df.head()

,numTrials,hour,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,uid_user_15,...,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1,dayofweek
0,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
2,3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
3,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
4,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4


In [5]:
X = df.drop('dayofweek', axis=1)
y = df['dayofweek']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

## 2. SVM

1. Use the best parameters from the previous exercise and train the model of SVM.
2. You need to calculate `accuracy`, `precision`, `recall`, `ROC AUC`.

 - `precision` and `recall` should be calculated for each class (use `average='weighted'`)
 - `ROC AUC` should be calculated for each class against any other class (all possible pairwise combinations) and then weighted average should be applied for the final metric
 - the code in the cell should display the result as below:

```
accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.97878
```

In [15]:
params = {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}
model = SVC(**params, probability=True)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

roc_auc = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr', average='weighted')

print(f'accuracy is {accuracy_score(y_test, y_pred)}')
print(f'precision is {precision_score(y_test, y_pred, average='weighted')}')
print(f'recall is {recall_score(y_test, y_pred, average='weighted')}')
print(f'roc_auc is {roc_auc}')


accuracy is 0.8875739644970414
precision is 0.8926729169690374
recall is 0.8875739644970414
roc_auc is 0.9820332974866085


## 3. Decision tree

1. The same task for decision tree

In [16]:
params = {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 23}
model = DecisionTreeClassifier(**params, random_state=21)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

roc_auc = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr', average='weighted')

print(f'accuracy is {accuracy_score(y_test, y_pred)}')
print(f'precision is {precision_score(y_test, y_pred, average='weighted')}')
print(f'recall is {recall_score(y_test, y_pred, average='weighted')}')
print(f'roc_auc is {roc_auc}')

accuracy is 0.893491124260355
precision is 0.8953094285678733
recall is 0.893491124260355
roc_auc is 0.9369245247365847


## 4. Random forest

1. The same task for random forest.

In [17]:
params = {'class_weight': None, 'criterion': 'gini', 'max_depth': 28, 'n_estimators': 50}
model = RandomForestClassifier(**params, random_state=21)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

roc_auc = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr', average='weighted')

print(f'accuracy is {accuracy_score(y_test, y_pred)}')
print(f'precision is {precision_score(y_test, y_pred, average='weighted')}')
print(f'recall is {recall_score(y_test, y_pred, average='weighted')}')
print(f'roc_auc is {roc_auc}')

accuracy is 0.9289940828402367
precision is 0.9300865038851309
recall is 0.9289940828402367
roc_auc is 0.9915076283905064


## 5. Predictions

1. Choose the best model.
2. Analyze: for which `weekday` your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which `labname` and for which `users`.
3. Save the model.

In [19]:
dayofweek_test = df.loc[X_test.index, 'dayofweek']
error_df = pd.DataFrame({
    'y_true': y_test,
    'y_pred': y_pred,
    'is_error': y_pred != y_test,
    'dayofweek': dayofweek_test
})


errors_by_day = error_df.groupby('dayofweek')['is_error'].sum()

total_by_day = df['dayofweek'].value_counts().sort_index()

day_stats = pd.DataFrame({
    'errors': errors_by_day,
    'total_samples': total_by_day
})

day_stats['error_percent'] = (day_stats['errors'] / day_stats['total_samples']) * 100

worst_day = day_stats['error_percent'].idxmax()
worst_error = day_stats.loc[worst_day, 'error_percent']

print(day_stats)
print(f"\nMost error-prone weekday: {worst_day} with {worst_error:.2f}% errors")

           errors  total_samples  error_percent
dayofweek                                      
0               7            136       5.147059
1               6            274       2.189781
2               2            149       1.342282
3               2            396       0.505051
4               3            104       2.884615
5               3            271       1.107011
6               1            356       0.280899

Most error-prone weekday: 0 with 5.15% errors


In [28]:
labname_cols = [col for col in df.columns if col.startswith('labname_')]

max_error_percent = 0
max_error_percent_col = None

for col in labname_cols:
    col_values_test = df.loc[X_test.index, col]

    error_df = pd.DataFrame({
        'y_true': y_test,
        'y_pred': y_pred,
        'is_error': y_pred != y_test,
        col: col_values_test
    })

    errors = error_df[error_df[col] == 1]['is_error'].sum()
    total = df[df[col] == 1].shape[0]

    error_percent = (errors / total) * 100 if total > 0 else 0

    if error_percent > max_error_percent:
        max_error_percent = error_percent
        max_error_percent_col = col

    print(f"{col}: {error_percent:.2f}% errors ({errors}/{total})")
    
print(f"Max error percent: {max_error_percent:.2f}% for {max_error_percent_col}")

labname_code_rvw: 1.22% errors (1/82)
labname_lab02: 0.00% errors (0/2)
labname_lab03: 100.00% errors (1/1)
labname_lab03s: 100.00% errors (1/1)
labname_lab05s: 2.78% errors (1/36)
labname_laba04: 3.37% errors (6/178)
labname_laba04s: 0.00% errors (0/104)
labname_laba05: 0.45% errors (1/222)
labname_laba06: 4.17% errors (2/48)
labname_laba06s: 3.28% errors (2/61)
labname_project1: 0.95% errors (9/951)
Max error percent: 100.00% for labname_lab03


In [31]:
user_cols = [col for col in df.columns if col.startswith('uid_user_')]

max_percent = 0
max_col = None

for col in user_cols:
    col_values_test = df.loc[X_test.index, col]

    error_df = pd.DataFrame({
        'y_true': y_test,
        'y_pred': y_pred,
        'is_error': y_pred != y_test,
        col: col_values_test
    })

    errors = error_df[error_df[col] == 1]['is_error'].sum()
    total = df[df[col] == 1].shape[0]

    error_percent = (errors / total) * 100 if total > 0 else 0

    if error_percent > max_percent:
        max_percent = error_percent
        max_col = col

    print(f"{col}: {error_percent:.2f}% errors ({errors}/{total})")

print(f'Max error percent: {max_percent:.2f}% ({max_col})')

uid_user_0: 0.00% errors (0/2)
uid_user_1: 0.00% errors (0/46)
uid_user_10: 1.41% errors (1/71)
uid_user_11: 0.00% errors (0/5)
uid_user_12: 0.00% errors (0/49)
uid_user_13: 0.00% errors (0/60)
uid_user_14: 0.76% errors (1/132)
uid_user_15: 0.00% errors (0/17)
uid_user_16: 6.25% errors (2/32)
uid_user_17: 0.00% errors (0/34)
uid_user_18: 2.86% errors (1/35)
uid_user_19: 4.40% errors (4/91)
uid_user_2: 0.83% errors (1/121)
uid_user_20: 0.00% errors (0/86)
uid_user_21: 0.00% errors (0/44)
uid_user_22: 14.29% errors (1/7)
uid_user_23: 0.00% errors (0/4)
uid_user_24: 1.79% errors (1/56)
uid_user_25: 1.67% errors (2/120)
uid_user_26: 0.00% errors (0/90)
uid_user_27: 4.35% errors (1/23)
uid_user_28: 0.00% errors (0/60)
uid_user_29: 1.56% errors (1/64)
uid_user_3: 2.82% errors (2/71)
uid_user_30: 2.56% errors (1/39)
uid_user_31: 2.67% errors (2/75)
uid_user_4: 1.06% errors (2/188)
uid_user_6: 8.33% errors (1/12)
uid_user_7: 0.00% errors (0/5)
uid_user_8: 0.00% errors (0/47)
Max error percent:

## 6. Function

1. Write a function that takes a list of different models and a corresponding list of parameters (dicts) and returns a dict that contains all the 4 metrics for each model.

In [40]:
def func(models, params):
    for model_name in models:
        model_class = eval(model_name)

        if model_name == 'SVC':
            model_result = model_class(**params[model_name], probability=True).fit(X_train, y_train)
        else:
            model_result = model_class(**params[model_name]).fit(X_train, y_train)
            
        y_pred = model_result.predict(X_test)
        print(f'{model_name} stats:')

        roc_auc = roc_auc_score(y_test, model_result.predict_proba(X_test), multi_class='ovr', average='weighted')

        print(f'accuracy is {accuracy_score(y_test, y_pred)}')
        print(f'precision is {precision_score(y_test, y_pred, average='weighted')}')
        print(f'recall is {recall_score(y_test, y_pred, average='weighted')}')
        print(f'roc_auc is {roc_auc}')  
        print('\n')

In [41]:
models_list = ['RandomForestClassifier', 'DecisionTreeClassifier', 'SVC']
models_params = {
    'RandomForestClassifier': {'class_weight': None, 'criterion': 'gini', 'max_depth': 28, 'n_estimators': 50},
    'DecisionTreeClassifier': {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 23},
    'SVC': {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}
}

func(models_list, models_params)

RandomForestClassifier stats:
accuracy is 0.9289940828402367
precision is 0.9320931192329144
recall is 0.9289940828402367
roc_auc is 0.9905356256858662


DecisionTreeClassifier stats:
accuracy is 0.9053254437869822
precision is 0.9064400481334297
recall is 0.9053254437869822
roc_auc is 0.9437356717893015


SVC stats:
accuracy is 0.8875739644970414
precision is 0.8926729169690374
recall is 0.8875739644970414
roc_auc is 0.9819078521045288


